# 0. Collecting The Data

## Uploading Modules

In [1]:
import os
import tweepy as tw
import pandas as pd
import datetime as dt

## Loading Twitter Data

In [2]:
consumer_key= 'LvRdHf5ngA2G8g0uEqJVJqNVC'
consumer_secret= '5PTWfqrpTf7FywHgjPfdHJiMGGzaKME24q9ubAB78nMvmIH71z'
access_token= '1194114471049887744-cWqTo2zFitH5qI3OmHcUCVqUD2v3f1'
access_token_secret= 'CE50qralUcPKnFhnb586p7dg5otsF96s3mngX2eKvBW2g'

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
users=pd.read_csv('Twitter users.csv')
users.head()

,user,type,keyword
0,prattprattpratt,active,outdoor
1,OutdoorPhotoMag,active,outdoor
2,timkaine,active,outdoor
3,BrooklynDecker,active,outdoor
4,micky_norcross,active,outdoor


In [5]:
active20_list = []
active19_list = []
start_date20=dt.datetime(2020, 3, 1, 0, 0, 0)
start_date19=dt.datetime(2019, 3, 1, 0, 0, 0)
end_date19=dt.datetime(2019, 4, 15, 0, 0, 0)

In [11]:
### make tweet lists for active users
active20_list = []
active19_list = []

for user in users.user[users.type=='active']:
    try: # block raising TweepError: Twitter error response: status code = 401 for private users
        tmpTweets = api.user_timeline(screen_name=user, include_rts=False, tweet_mode='extended')
        for tweet in tmpTweets:
            if tweet.created_at > start_date20:
                array = [tweet._json["user"],tweet._json["full_text"].strip(), tweet._json["created_at"]]
                active20_list.append(array)
            if tweet.created_at > start_date19 and tweet.created_at < end_date19:
                array = [tweet._json["user"],tweet._json["full_text"].strip(), tweet._json["created_at"]]
                active19_list.append(array)
    except:
        pass

In [6]:
### make tweet lists for lazy users
lazy20_list = []
lazy19_list = []

for user in users.user[users.type=='lazy']:
    try: # block raising TweepError: Twitter error response: status code = 401 for private users
        tmpTweets = api.user_timeline(screen_name=user, include_rts=False, tweet_mode='extended')
        for tweet in tmpTweets:
            if tweet.created_at > start_date20:
                array = [tweet._json["user"],tweet._json["full_text"].strip(), tweet._json["created_at"]]
                lazy20_list.append(array)
            if tweet.created_at > start_date19 and tweet.created_at < end_date19:
                array = [tweet._json["user"],tweet._json["full_text"].strip(), tweet._json["created_at"]]
                lazy19_list.append(array)
    except:
        pass

In [ ]:
for user in users.user[users.type=='lazy']:
    try:
        tmpTweets = tw.Cursor(api.user_timeline, since = start_date19, until = end_date19, id=user,
              include_rts=False, tweet_mode='extended').items(1000)
        for tweet in tmpTweets:
            if tweet._json["created_at"] > start_date19:
                array = [tweet._json["user"],tweet._json["full_text"].strip(), tweet._json["created_at"]]
                lazy19_list.append(array)
            else:
                pass
    except:
        pass

In [7]:
# create a function to make a dataframe in the format we need:
def make_df(my_list):
    my_df = pd.DataFrame(my_list, columns=['screen_name',"Text","Created at"])
    my_df=my_df.drop(my_df.index[(my_df.Text.str.startswith('RT'))|(my_df.Text.str.startswith('@'))],axis=0)
    my_df=my_df.reset_index()
    for i in range(0, len(my_df)):
        my_df.loc[i,'Date'] = pd.to_datetime(my_df.loc[i,'Created at'], format='%a %b %d %H:%M:%S +0000 %Y')
        my_df.loc[i,'Name'] = my_df.loc[i,'screen_name']['screen_name']
        my_df.loc[i,'Location'] = my_df.loc[i,'screen_name']['location']

    my_df=my_df.drop(['index','screen_name','Created at'],axis=1)
    my_df=my_df.reset_index()
    return(my_df)

In [29]:
#active20=make_df(active20_list)
#active19=make_df(active19_list)
#lazy20=make_df(lazy20_list)
lazy19_2=make_df(lazy19_list)

In [31]:
lazy19_2.reset_index().to_csv("lazy19_2.csv")

## Files for Future Use

In [9]:
#save dataframes as csv for future analysis
active20.reset_index().to_csv("active20.csv")
active19.reset_index().to_csv("active19.csv")
lazy20.reset_index().to_csv("lazy20.csv")
lazy19.reset_index().to_csv("lazy19.csv")